# Data Visualization

This is a notebook for the final project of Fundamentals of Data Visualization course



In [204]:
import pandas as pd
import numpy as np

data= pd.read_csv("NIFTY50.csv")
#data['Date']= pd.to_datetime(data['Date']).dt.date
data['Date']= pd.to_datetime(data['Date'])
print(len(data))


5474


In [205]:
max_pe = data['P/E'].max()
min_pe = data['P/E'].min()
mid_pe = int((max_pe+min_pe)/2)
margin = 2
mean_pe = round(data['P/E'].mean())
range_min_pe = mean_pe -margin
range_max_pe = mean_pe+margin
data['affordability'] = np.where(data['P/E']< range_max_pe, 'cheap', 'costly')

print(f'max {max_pe} min {min_pe} mid {mid_pe} mean {mean_pe}')

max 42.0 min 10.68 mid 26 mean 21


In [206]:
data.tail()

,Date,Open,High,Low,Close,P/E,affordability
5469,2021-12-27,16937.75,17112.05,16833.20,17086.25,23.57,costly
5470,2021-12-28,17177.60,17250.25,17161.15,17233.25,23.77,costly
5471,2021-12-29,17220.10,17285.95,17176.65,17213.60,23.74,costly
5472,2021-12-30,17201.45,17264.05,17146.35,17203.95,23.73,costly
5473,2021-12-31,17244.50,17400.80,17238.50,17354.05,24.11,costly


In [ ]:
# Now let's visualize the data
import altair as alt
import datetime

alt.data_transformers.disable_max_rows()
domain = ['costly', 'cheap']
range_ = ['red', 'green']



alt.Chart(data).mark_line().encode(
    x='Date',
    y=alt.Y('P/E', impute=alt.ImputeParams(value=None)),
    color=alt.Color('affordability', scale=alt.Scale(domain=domain, range=range_))
    
)


In [208]:
from dateutil.relativedelta import relativedelta
ser = data['Date']
yr_list = dict()
for tgt_yr in [1,3,5,7,10,15,20]:
    ret_list = list()
    for i in range(len(data)):
        #print(f'{data.iloc[[0]]}')
        curr_row = data.iloc[[i]]
        try:
            dt = curr_row['Date'][i]
        except Exception as ex:
            print(f'exception {ex} when getting date using {curr_row} and {curr_row["Date"]}')
        #print(f'dt {dt}')

        target_dt = dt+relativedelta(years=tgt_yr)
        #print(f'target_dt {target_dt}')

        index = ser.searchsorted(target_dt)
        if index >= len(data):
            break
        target_row = data.iloc[index]
        #print(f'{dt}: {curr_row["Close"]} {target_row["Date"][0]}:{target_row["Close"][0]} ')
        start = curr_row["Close"][i]
        end = target_row["Close"]
        roi = (end-start)*100/start
        #returns = round(roi)
        if tgt_yr == 1:
            returns = round(roi)
        else:
            #returns = ((1+end-start)**(1/tgt_yr))-1
            returns=(pow((end/start),(1/tgt_yr))-1)*100
        #print(f'{dt}: {curr_row["Close"][0]}  {target_row["Date"]}: {target_row["Close"]} returns:{returns}')

        if np.isnan(returns):
            print('nan')
        else:
            ret_list.append({'date':dt, 'target_dt':target_row['Date'], 'close_now':start, 'close_target':end, 'affordability':curr_row['affordability'][i], 'returns':round(returns)})
    yr_list[tgt_yr] = ret_list
#print(f'returns_yr: {returns_yr}')
df = pd.DataFrame(yr_list[1])

In [209]:
df.head()

,date,target_dt,close_now,close_target,affordability,returns
0,2000-01-03,2001-01-03,1592.2,1291.25,costly,-19
1,2000-01-04,2001-01-04,1638.7,1307.65,costly,-20
2,2000-01-05,2001-01-05,1595.8,1327.25,costly,-17
3,2000-01-06,2001-01-08,1617.6,1309.25,costly,-19
4,2000-01-07,2001-01-08,1613.3,1309.25,costly,-19


In [ ]:
alt.Chart(df).mark_circle().encode(x="date", y="returns", color="affordability")

In [ ]:
df = pd.DataFrame(yr_list[3])
alt.Chart(df).mark_circle().encode(x="date", y="returns", color="affordability")

In [ ]:
df = pd.DataFrame(yr_list[15])
alt.Chart(df).mark_circle().encode(x="date", y="returns", color="affordability")

In [ ]:
df = pd.DataFrame(yr_list[20])
alt.Chart(df).transform_density(
    'returns',
    as_=['returns', 'density'],
    extent=[5, 20],
    groupby=['affordability']
).mark_area(orient='horizontal').encode(
    y='returns:Q',
    color='affordability:N',
    x=alt.X(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
    ),
    column=alt.Column(
        'affordability:N',
        header=alt.Header(
            titleOrient='bottom',
            labelOrient='bottom',
            labelPadding=0,
            title='Affordability'
        ),
    )
).properties(
    width=100
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)

In [ ]:
df = pd.DataFrame(yr_list[1])
alt.Chart(df).transform_density(
    'returns',
    as_=['returns', 'density'],
    extent=[-50, 150],
    groupby=['affordability']
).mark_area(orient='horizontal').encode(
    y='returns:Q',
    color='affordability:N',
    x=alt.X(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
    ),
    column=alt.Column(
        'affordability:N',
        header=alt.Header(
            titleOrient='bottom',
            labelOrient='bottom',
            labelPadding=0,
            title='Affordability'
        ),
    )
).properties(
    width=100
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)